In [35]:
%pip install groq

In [36]:
from dotenv import load_dotenv
import os
from groq import Groq


os.environ['GROQ_API_KEY'] = "YOUR_GROQ_API_KEY_HERE"
load_dotenv()

In [37]:
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

chat_completion = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "Explain the importance of fast language models"}
    ],
    model="llama-3.1-8b-instant",
    temperature=0
)

print(chat_completion.choices[0].message.content)

Fast language models have gained significant attention in recent years due to their ability to process and generate human-like text quickly and efficiently. The importance of fast language models can be understood from several perspectives:

1. **Real-time Applications**: Fast language models are crucial for real-time applications such as chatbots, virtual assistants, and language translation systems. They enable these systems to respond quickly to user queries, providing a seamless and interactive experience.
2. **Scalability**: As the demand for language-based services grows, fast language models can handle a large volume of requests without compromising performance. This scalability is essential for businesses and organizations that rely on language-based services.
3. **Efficient Resource Utilization**: Fast language models can run on lower-end hardware, reducing the need for expensive and power-hungry computing resources. This makes them more accessible to organizations with limite

In [38]:
class Agent:
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="llama-3.1-8b-instant", messages=self.messages
        )
        return completion.choices[0].message.content

In [39]:
system_prompt = """
You are a Corporate Meeting Assistant that helps schedule meetings in a professional environment.
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.

Use Thought to describe your thoughts about the scheduling request.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

check_availability:
e.g. check_availability: 2024-01-15 14:00
Checks if the specified date and time slot is available in the calendar.
Format: YYYY-MM-DD HH:MM (24-hour format)
Returns whether the slot is available or occupied.

schedule_meeting:
e.g. schedule_meeting: 2024-01-15 14:00, Team Planning Session, 1 hour
Schedules a meeting at the specified date/time with title and duration.
Format: date time, meeting title, duration
Only use this AFTER confirming availability with check_availability.

send_email:
e.g. send_email: Meeting scheduled for 2024-01-15 at 14:00 - Team Planning Session
Sends an email notification to all employees about the scheduled meeting.
Include date, time, and meeting title in the message.

Example session:

Question: Schedule a team meeting for tomorrow at 2 PM
Thought: I need to check if tomorrow at 2 PM is available, then schedule the meeting and notify everyone.
Action: check_availability: 2024-01-16 14:00
PAUSE

You will be called again with this:

Observation: Available - no conflicts found

Thought: The slot is available, now I'll schedule the meeting.
Action: schedule_meeting: 2024-01-16 14:00, Team Meeting, 1 hour
PAUSE

You will be called again with this:

Observation: Meeting scheduled successfully - Team Meeting on 2024-01-16 at 14:00

Thought: Now I need to notify all employees about the meeting.
Action: send_email: Meeting scheduled for 2024-01-16 at 14:00 - Team Meeting
PAUSE

You will be called again with this:

Observation: Email sent to 12 employees successfully

Answer: Successfully scheduled Team Meeting for 2024-01-16 at 14:00 and notified all employees via email.

Important notes:
- Always check availability BEFORE scheduling
- Always send email notification AFTER scheduling
- Use 24-hour format for times (14:00 instead of 2 PM)
- Include meeting title in communications

Now it's your turn:
""".strip()

# MOCKS para simular um ambiente coorporativo

# Lista de funcionários da empresa
EMPLOYEES = [
    {"name": "Joao", "email": "joao@empresa.com", "department": "TI"},
    {"name": "Carlos", "email": "carlos@empresa.com", "department": "RH"},
    {"name": "Marina", "email": "marina@empresa.com", "department": "Marketing"},
]

OCCUPIED_SLOTS = [
    {"date": "2025-08-29", "time": "09:00", "title": "Alinhamento Semanal", "duration": "2 hours"},
    {"date": "2025-09-01", "time": "14:00", "title": "Discussão sobre novas implementações", "duration": "1 hour"},
    {"date": "2025-08-29", "time": "13:30", "title": "Fechamento Mensal", "duration": "1.5 hours"},
    {"date": "2025-09-01", "time": "15:00", "title": "Comemoração de Aniversário", "duration": "30 minutos"},

]

# Lista de reuniões agendadas (será atualizada durante execução)
SCHEDULED_MEETINGS = []

# NOVAS FERRAMENTAS CORPORATIVAS

def check_availability(datetime_slot: str) -> str:
    """
    Verifica se o horário está disponível no calendário
    Retorna uma mensagem indicando se está disponível ou ocupado
    """
    try:
        # Juntar o par DATA/HORA
        date_part, time_part = datetime_slot.strip().split(" ")

        # Verifica se já existe compromisso nesse horário
        for slot in OCCUPIED_SLOTS:
            if slot["date"] == date_part and slot["time"] == time_part:
                return f"Ocupado - {slot['title']} já agendada às {time_part} de {date_part}"

        # Verifica se já foi agendado nesta sessão
        for meeting in SCHEDULED_MEETINGS:
            if meeting["date"] == date_part and meeting["time"] == time_part:
                return f"Ocupado - {meeting['title']} já agendada nesta sessão"

        return f"Disponível - Horário livre em {date_part} às {time_part}"

    except Exception as e:
        return f"Erro ao verificar disponibilidade: {e}"

def schedule_meeting(meeting_details: str) -> str:
    """
    Agenda uma reunião no calendário quando possível
    Retorna uma mensagem confirmando (ou não) o agendamento
    """
    try:
        # Agrupar os detalhes da reunião
        parts = meeting_details.strip().split(", ")
        if len(parts) < 3:
            return "Formato inválido. Use: YYYY-MM-DD HH:MM, Título, Duração"

        datetime_slot = parts[0].strip()
        title = parts[1].strip()
        duration = parts[2].strip()

        date_part, time_part = datetime_slot.split(" ")

        # Verifica se está disponível antes de agendar
        availability = check_availability(datetime_slot)
        if "Ocupado" in availability:
            return f"Não foi possível agendar - {availability}"

        # Agenda a reunião
        new_meeting = {
            "date": date_part,
            "time": time_part,
            "title": title,
            "duration": duration,
            "attendees": len(EMPLOYEES)
        }

        SCHEDULED_MEETINGS.append(new_meeting)

        return f"Reunião agendada com sucesso - '{title}' em {date_part} às {time_part} ({duration})"

    except Exception as e:
        return f"Erro ao agendar reunião: {e}"

def send_email(message: str) -> str:
    """
    Envia email para todos os funcionários sobre a reunião
    Retorna uma mensagem confirmando o envio do email
    """
    try:
        # Envio de email para todos os funcionários
        email_count = len(EMPLOYEES)

        # Extrai informações da mensagem para criar um email mais detalhado
        email_body = f""" NOVA REUNIÃO AGENDADA

        {message}

        Detalhes:
        - Participantes: Todos os funcionários ({email_count} pessoas)
        - Local: Sala de Reuniões Principal

        *Este é um email automático do sistema de agendamento.*

        Atenciosamente,
        Agente Corporativo
        """

        # Simula log de envio
        recipients = [emp["email"] for emp in EMPLOYEES]

        return f"Email enviado com sucesso para {email_count} funcionários ({', '.join(recipients[:3])}...)"

    except Exception as e:
        return f"Erro ao enviar email: {e}"

In [40]:
agente_coorporativo = Agent(client=client, system=system_prompt)

In [41]:
result = agente_coorporativo(message="Schedule a team meeting for 2024-01-17 at 14:00")
print(result)

Thought: I need to check if 2024-01-17 at 2 PM is available, then schedule the meeting and notify everyone.
Action: check_availability: 2024-01-17 14:00
PAUSE


In [42]:
agente_coorporativo.messages

[{'role': 'system',
  'content': "You are a Corporate Meeting Assistant that helps schedule meetings in a professional environment.\nYou run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer.\n\nUse Thought to describe your thoughts about the scheduling request.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncheck_availability:\ne.g. check_availability: 2024-01-15 14:00\nChecks if the specified date and time slot is available in the calendar. \nFormat: YYYY-MM-DD HH:MM (24-hour format)\nReturns whether the slot is available or occupied.\n\nschedule_meeting:\ne.g. schedule_meeting: 2024-01-15 14:00, Team Planning Session, 1 hour\nSchedules a meeting at the specified date/time with title and duration.\nFormat: date time, meeting title, duration\nOnly use this AFTER confirming availability with check_availability.\n\nsend_ema

In [43]:
result = check_availability("2024-01-17 14:00")
print(result)

Disponível - Horário livre em 2024-01-17 às 14:00


In [44]:
next_prompt = "Observation: {}".format(result)
next_prompt

'Observation: Disponível - Horário livre em 2024-01-17 às 14:00'

In [45]:
result = agente_coorporativo(next_prompt)
print(result)

Thought: The slot is available, now I'll schedule the meeting.
Action: schedule_meeting: 2024-01-17 14:00, Team Meeting, 1 hour
PAUSE


In [46]:
result = schedule_meeting("2024-01-17 14:00, Team Meeting, 1 hour")
print(result)

Reunião agendada com sucesso - 'Team Meeting' em 2024-01-17 às 14:00 (1 hour)


In [47]:
next_prompt = "Observation: {}".format(result)
next_prompt

"Observation: Reunião agendada com sucesso - 'Team Meeting' em 2024-01-17 às 14:00 (1 hour)"

In [48]:
result = agente_coorporativo(next_prompt)
print(result)

Thought: Now I need to notify all employees about the meeting.
Action: send_email: Meeting scheduled for 2024-01-17 at 14:00 - Team Meeting
PAUSE


In [49]:
result = send_email("2024-01-17 14:00, Team Meeting, 1 hour")
print(result)

Email enviado com sucesso para 3 funcionários (joao@empresa.com, carlos@empresa.com, marina@empresa.com...)


# INÍCIO DO LOOP

In [57]:
import re


def loop(max_iterations=10, query: str = ""):

    agent = Agent(client=client, system=system_prompt)

    tools = ["check_availability", "schedule_meeting", "send_email"]  # ALTERADO: novas ferramentas

    next_prompt = query

    i = 0

    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"

            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue

        if "Answer" in result:
            break

In [54]:
loop(query="Schedule a team meeting for 2024-01-17 at 14:00") # Horário já AGENDADO

Thought: I need to check if 2024-01-17 at 14:00 is available, then schedule the meeting and notify everyone.
Action: check_availability: 2024-01-17 14:00

PAUSE
Observation: Ocupado - Team Meeting já agendada nesta sessão
Thought: The slot is occupied, which means there's already a meeting scheduled for that time.
Action: schedule_meeting: 2024-01-17 14:00, Team Meeting, 2 hour

PAUSE
Observation: Não foi possível agendar - Ocupado - Team Meeting já agendada nesta sessão
Thought: It looks like the team meeting has already been scheduled for 2 hours. I should try rescheduling it for a later date.
Action: check_availability: 2024-01-18 10:00 PAUSE
Observation: Erro ao verificar disponibilidade: too many values to unpack (expected 2)
It seems that there's an error in my action. The check_availability function was called with two extra values. Let me correct it.

Thought: I need to reschedule the meeting for 2024-01-18 at 10:00.
Action: check_availability: 2024-01-18 10:00 

PAUSE
Observat

In [58]:
loop(query="Schedule a team meeting for 2025-09-17 at 14:00") # Horário LIVRE

Thought: I need to check if 2025-09-17 at 14:00 is available, then schedule the meeting and notify everyone.
Action: check_availability: 2025-09-17 14:00
PAUSE
Observation: Available - Time slot free on 2025-09-17 at 14:00
Thought: The slot is available, now I'll schedule the meeting.
Action: schedule_meeting: 2025-09-17 14:00, Team Meeting, 1 hour
PAUSE
Observation: Meeting scheduled successfully - 'Team Meeting' on 2025-09-17 at 14:00 (1 hour)
Thought: Now I need to notify all employees about the meeting.
Action: send_email: Meeting scheduled for 2025-09-17 at 14:00 - Team Meeting
PAUSE
Observation: Email sent successfully to 3 employees
Answer: Successfully scheduled Team Meeting for 2025-09-17 at 14:00 and notified 3 employees via email.


# Detecção de Erros + Novas Alterações

Após alguns testes, notei que alguns detalhes estavam comprometendo o funcionamento correto do Agente, as novas alterações realizadas envolvem:



1.   Melhoria no Prompt: Maior especificação a respeito do uso das ferramentas, pontuando a ordem das ações a serem seguidas
2.   Tradução completa para o Inglês: Algumas respostas em português pareciam estar confundindo o Agente





In [59]:
system_prompt = """
You are a Corporate Meeting Assistant that helps schedule meetings in a professional environment.
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer

Use Thought to describe your thoughts about the scheduling request.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

check_availability:
e.g. check_availability: 2024-01-15 14:00
Checks if the specified date and time slot is available in the calendar.
Format: YYYY-MM-DD HH:MM (24-hour format)

schedule_meeting:
e.g. schedule_meeting: 2024-01-15 14:00, Team Meeting, 1 hour
Schedules a meeting at the specified date/time with title and duration.
Format: date time, meeting title, duration

send_email:
e.g. send_email: Meeting scheduled for 2024-01-15 at 14:00 - Team Meeting
Sends an email notification to all employees about the scheduled meeting.

Example session:

Question: Schedule a team meeting for tomorrow at 2 PM
Thought: I need to check if tomorrow at 2 PM is available, then schedule the meeting and notify everyone.
Action: check_availability: 2024-01-16 14:00
PAUSE

You will be called again with this:

Observation: Available - Time slot free on 2024-01-16 at 14:00

Thought: The slot is available, now I'll schedule the meeting.
Action: schedule_meeting: 2024-01-16 14:00, Team Meeting, 1 hour
PAUSE

You will be called again with this:

Observation: Meeting scheduled successfully - 'Team Meeting' on 2024-01-16 at 14:00 (1 hour)

Thought: Now I need to notify all employees about the meeting.
Action: send_email: Meeting scheduled for 2024-01-16 at 14:00 - Team Meeting
PAUSE

You will be called again with this:

Observation: Email sent successfully to 12 employees

Answer: Successfully scheduled Team Meeting for 2024-01-16 at 14:00 and notified all employees via email.

IMPORTANT: Always follow this order:
1. FIRST check_availability
2. THEN schedule_meeting
3. FINALLY send_email

Now it's your turn:
""".strip()

# MOCKS para simular ambiente corporativo
EMPLOYEES = [
    {"name": "Joao", "email": "joao@empresa.com", "department": "TI"},
    {"name": "Carlos", "email": "carlos@empresa.com", "department": "RH"},
    {"name": "Marina", "email": "marina@empresa.com", "department": "Marketing"},
]

OCCUPIED_SLOTS = [
    {"date": "2025-08-29", "time": "09:00", "title": "Alinhamento Semanal", "duration": "2 hours"},
    {"date": "2025-09-01", "time": "14:00", "title": "Discussão sobre novas implementações", "duration": "1 hour"},
    {"date": "2025-08-29", "time": "13:30", "title": "Fechamento Mensal", "duration": "1.5 hours"},
    {"date": "2025-09-01", "time": "15:00", "title": "Comemoração de Aniversário", "duration": "30 minutos"},

]

SCHEDULED_MEETINGS = []

def check_availability(datetime_slot: str) -> str:

    try:
        date_part, time_part = datetime_slot.strip().split(" ")

        for slot in OCCUPIED_SLOTS:
            if slot["date"] == date_part and slot["time"] == time_part:
                return f"Occupied - {slot['title']} already scheduled"

        for meeting in SCHEDULED_MEETINGS:
            if meeting["date"] == date_part and meeting["time"] == time_part:
                return f"Occupied - {meeting['title']} already scheduled"

        return f"Available - Time slot free on {date_part} at {time_part}"
    except Exception as e:
        return f"Error checking availability: {e}"

def schedule_meeting(meeting_details: str) -> str:

    try:
        parts = meeting_details.strip().split(", ")
        if len(parts) < 3:
            return "Invalid format. Use: YYYY-MM-DD HH:MM, Title, Duration"

        datetime_slot = parts[0].strip()
        title = parts[1].strip()
        duration = parts[2].strip()
        date_part, time_part = datetime_slot.split(" ")

        availability = check_availability(datetime_slot)
        if "Occupied" in availability:
            return f"Cannot schedule - {availability}"

        new_meeting = {
            "date": date_part,
            "time": time_part,
            "title": title,
            "duration": duration,
            "attendees": len(EMPLOYEES)
        }

        SCHEDULED_MEETINGS.append(new_meeting)
        return f"Meeting scheduled successfully - '{title}' on {date_part} at {time_part} ({duration})"
    except Exception as e:
        return f"Error scheduling meeting: {e}"

def send_email(message: str) -> str:

    try:
        email_count = len(EMPLOYEES)
        recipients = [emp["email"] for emp in EMPLOYEES]
        return f"Email sent successfully to {email_count} employees"
    except Exception as e:
        return f"Error sending email: {e}"

In [56]:
loop(query="Schedule a team meeting for 2025-09-01 at 14:00") # Horário já AGENDADO

Thought: I need to check if 2025-09-01 at 14:00 is available, then schedule the meeting and notify everyone.
Action: check_availability: 2025-09-01 14:00
PAUSE
Observation: Occupied - Discussão sobre novas implementações already scheduled
Thought: The slot is occupied, so I now need to schedule the new meeting.
Action: schedule_meeting: 2025-09-01 14:00, Team Meeting, 1 hour
PAUSE
Observation: Cannot schedule - Occupied - Discussão sobre novas implementações already scheduled
Thought: The requested time slot is completely taken, I need to look for an alternative time for the team meeting.
However, following the protocol I should not proceed with new actions until further instruction.
Thought: The meeting slot remains occupied and scheduling the meeting is not possible.
Action: None
No action can be taken as per the scheduling protocol until an alternative time slot is provided or a change in the existing schedule occurs.

Answer: Cannot schedule Team Meeting for 2025-09-01 at 14:00 due

### Interessantemente, após as melhorias no prompt, o Agente passou a parar de tentar reagendar a reunião para um novo horário disponível e agora está indicando que não poderá realizar o agendamento, conforme desejávamos

In [60]:
loop(query="Schedule a team meeting for 2025-09-03 at 14:00") # Horário LIVRE

Thought: I need to check if 2025-09-03 at 14:00 is available, then schedule the meeting and notify everyone.

Action: check_availability: 2025-09-03 14:00
PAUSE
Observation: Available - Time slot free on 2025-09-03 at 14:00
Thought: The slot is available, now I'll schedule the meeting for 1 hour.

Action: schedule_meeting: 2025-09-03 14:00, Team Meeting, 1 hour
PAUSE
Observation: Meeting scheduled successfully - 'Team Meeting' on 2025-09-03 at 14:00 (1 hour)
Thought: Now I need to notify all employees about the meeting.

Action: send_email: Meeting scheduled for 2025-09-03 at 14:00 - Team Meeting
PAUSE
Observation: Email sent successfully to 3 employees
Answer: Successfully scheduled Team Meeting for 2025-09-03 at 14:00 and notified all employees via email.
